In [1]:
import tensorflow as tf

In TensorFlow the computations are a represented in the form of graphs. Here, graphs are the descriptions of the computations which will be executed when the graphs are launched in a session. Before this launch they are just a “representation” of how the data will flow and what computations will be performed upon this data. Designing a graph is equivalent to designing a solution to a mathematical or a real world problem. Later, we can feed some input data to these graphs and get the desired output data. Nodes in the graphs are called ops (short for operations). These ops again are just representations of the real computations which would occur when the graphs are launched in a session and some data is provided. Here, data is represented by tensors. Tensors are the symbolic handle to the inputs and outputs of the ops. The real concrete values are stored in the form numpy arrays. Tensor does not hold the real values but instead provides a means of computing those values in a TensorFlow Session.

## There are three core Graph data structures, which are essential in designing TensorFlow Graphs:
class tf.Graph
class tf.Operation
class tf.Tensor


The Graph is used to represent a complex TensorFlow computation or a model design solution for a problem. This class contains a set of Tensor objects, which represent the units of data that flow between operations and a set of Operation objects, which represent units of computation. These objects are used to design a Graph object. A Graph is a fundamental requirement of executing a TensorFlow computation. Hence, a default Graph is always registered, and accessible by calling tf.get_default_graph(). To add an operation to the default graph, we can simply call one of the functions that defines a new Operation and this operation will be added as a unit of this default Graph. Hence, for a particular process all the operations and tensors defined will be added to the default graph unless we create a graph explicitly and add units to it.

In [2]:
# Build a graph. Here, the default graph
a = tf.constant(5.0)
b = tf.constant(6.0)
c = tf.mul(a, b)

# Launch the graph in a session.
sess = tf.Session()

# Evaluate the tensor `c`.
print(sess.run(c))
sess.close()

# Using context manager.

with tf.Session() as sess:
    print(sess.run(c))
    
    
print("Default Graph Object: ", tf.get_default_graph())
print("Node \'a\' added to the Graph Obect: ", a.graph)

if tf.get_default_graph() == a.graph:
    print('Tensor Object \'a\' added to the Default Graph object')




30.0
30.0
Default Graph Object:  <tensorflow.python.framework.ops.Graph object at 0x7f1a2d2b7160>
Node 'a' added to the Graph Obect:  <tensorflow.python.framework.ops.Graph object at 0x7f1a2d2b7160>
Tensor Object 'a' added to the Default Graph object


Also, we can create multiple graphs in the same process. For convenience, a global default graph is provided, and all ops will be added to this default graph. But we can create graphs explicitly using tf.Graph.as_default() which Returns a context manager that makes this Graph the default graph for the lifetime of the context. Using this method with the with keyword we can specify that ops created within the scope of a block should be added to this Graph.

In [3]:
print(tf.get_default_graph())

a = tf.constant(1.0)
b = tf.constant(10.0)
c = tf.mul(a, b)

with tf.Session() as sess:
    print(sess.run(c))



g1 = tf.Graph()
with g1.as_default():
    print(tf.get_default_graph())
    p = tf.constant(5.0)
    q = tf.constant(4.0)
    r = tf.mul(p, q)
    
with tf.Session(graph = g1) as sess:  
    real_r_output =  sess.run(r)
    print(real_r_output)
    

g2 = tf.Graph()
with g2.as_default():
    print(tf.get_default_graph())
    x = tf.constant(5.0)
    y = tf.constant(6.0)
    z = tf.mul(x, y)  
    
        
with tf.Session(graph = g2) as sess:  
        real_z_output =  sess.run(z)
        print(real_z_output)



10.0
20.0
30.0


## Reseting the Default Graph:

tf.reset_default_graph() function can be used to clear all the nodes and ops added to the default graph.

The default graph is a property of the current thread. This function applies only to the current thread. 

We can not use any previously created tf.Operation or tf.Tensor objects after calling this function as they will no longer exist for the current thread.

Example:

In [4]:
a = tf.constant(1.0)
b = tf.constant(10.0)
c = tf.mul(a, b)

tf.reset_default_graph()

with tf.Session() as sess:
    try:
        print(sess.run(c)) # Error
    except RuntimeError as e:
        print (e)

The Session graph is empty.  Add operations to the graph before calling run().


## Saving and Using Graph Definition:

TensorFlow provides a serialization method to save the graph definiton of a graph and further use of this definition in another Graph. This tool can be used to link multiple graphs.

In [5]:

g1 = tf.Graph()
with g1.as_default():

    p = tf.placeholder(tf.float32, name="value1")
    q = tf.placeholder(tf.float32, name="value2")
    r = tf.mul(p, q)
    product = tf.identity(r, name="product")

g1_def = g1.as_graph_def()


g2 = tf.Graph()

with g2.as_default():
    p = tf.constant(4.0)
    q = tf.constant(5.0)
    product = tf.import_graph_def(g1_def,input_map={"value1:0":p, "value2:0":q},
                                  return_elements=["product:0"])
    
with tf.Session(graph = g2) as sess:          
        print(sess.run(product))



[20.0]


## Control Dependencies:

TensorFlow provides addidng control dependency between two or more ops. If we want one operation to be dependent on another operation we can easily add the dependency to the operations. 

tf.Graph.control_dependencies(control_inputs) can be used to return a context manager that specifies control dependencies.

Use with the with keyword to specify that all operations constructed within the context should have control dependencies on control_inputs.

For Example:

In [6]:
tf.reset_default_graph()
a = tf.constant(2.0)
b = tf.constant(10.0)

def execute_op_first(a,b):
        op_first = tf.add(a,b)
        return op_first


op_first = execute_op_first(a,b)
with tf.get_default_graph().control_dependencies([op_first]):
        
        op_second = tf.mul(a, b)
        

        with tf.Session(graph = tf.get_default_graph()) as sess:          
              print(sess.run(op_second))

20.0


## Name Scopes:

In TensorFlow graph maintains a stack of name scopes.

Name Scopes are most important in data visualization. TensorBoard uses name scopes for this purpose. The better your name scopes, the better your visualization.

A with name_scope(...): statement can be used to push a new name onto the stack for the lifetime of the context.

The name argument will be interpreted as follows:

A string (not ending with '/') will create a new name scope, in which name is appended to the prefix of all operations created in the context. If name has been used before, it will be made unique by calling self.unique_name(name).

A scope previously captured from a with g.name_scope(...) as scope: statement will be treated as an "absolute" name scope, which makes it possible to re-enter existing scopes.

A value of None or the empty string will reset the current name scope to the top-level (empty) name scope.

For example:


In [7]:
with tf.Graph().as_default() as g:
            a = tf.constant(5.0, name="a")
            print('a: ', a.op.name)
            a = tf.constant(6.0, name="a")
            print('a: ', a.op.name)


                  # Creates a scope called "nested"
            with g.name_scope("layer") as scope:
                    b = tf.constant(10.0, name="b")
                    print ('b: ', b.op.name)

                    # Creates a nested scope called "inner".
                    with g.name_scope("inner"):
                        c = tf.constant(20.0, name="c")
                        print('c: ', c.op.name)

                    # Create a nested scope called "inner_1".
                    with g.name_scope("inner"):
                        c = tf.constant(30.0, name="c")
                        print('c: ', c.op.name)
 
                      # Treats `scope` as an absolute name scope, and
                      # switches to the "nested/" scope.
                    with g.name_scope(scope):
                        d = tf.constant(40.0, name="d")
                        print('d: ', d.op.name)

                        with g.name_scope(""):
                            e = tf.constant(50.0, name="e")
                            print('e: ',e.op.name)

a:  a
a:  a_1
b:  layer/b
c:  layer/inner/c
c:  layer/inner_1/c
d:  layer/d
e:  e


## Using Graph Collections:

Tensorflow is a system for specifying and then executing computational data flow graphs. The graph collections are used as part of keeping track of the constructed graphs and how they must be executed.

We can add specific type of values to a given key using tf.Graph.add_to_collection(name, value) function.

For example: all the Variables of a graph 'g1' are stored under the name 'VARIABLES' in the code below.


These names or keys anything. But in general these are standard names. The GraphKeys class contains many standard names for collections.

Standard names to use for graph collections.

The standard library uses various well-known names to collect and retrieve values associated with a graph. For example, the tf.Optimizer subclasses default to optimizing the variables collected under tf.GraphKeys.TRAINABLE_VARIABLES if none is specified, but it is also possible to pass an explicit list of variables.

The following standard keys are defined:

VARIABLES: the Variable objects that comprise a model, and must be saved and restored together.

TRAINABLE_VARIABLES: the subset of Variable objects that will be trained by an optimizer.

SUMMARIES: the summary Tensor objects that have been created in the graph.

QUEUE_RUNNERS: the QueueRunner objects that are used to produce input for a computation.

MOVING_AVERAGE_VARIABLES: the subset of Variable objects that will also keep moving averages.

REGULARIZATION_LOSSES: regularization losses collected during graph construction.

WEIGHTS: weights inside neural network layers.

BIASES: biases inside neural network layers.

ACTIVATIONS: activations of neural network layers.



These collection of values can later be accessed using tf.Graph.get_collection(key) function.



In [8]:
g1 = tf.Graph()
with g1.as_default():
    p = tf.Variable(1.0, name="value1")
    q = tf.Variable(2.0, name="value2")
    r = tf.mul(p, q)
    product = tf.identity(r, name="product")
    
    g1.add_to_collection('VARIABLES', p)
    g1.add_to_collection('VARIABLES', q)
    g1.add_to_collection('TRAINABLE_VARIABLES', r)
    
    
g2 = tf.Graph() 
with g2.as_default():
    p = tf.Variable(1.0, name="value1")
    q = tf.Variable(2.0, name="value2")
    s = tf.Variable(3.0, name="value3")
    r = tf.mul(p, q)
    product = tf.identity(r, name="product")
    
    g2.add_to_collection('VARIABLES', p)
    g2.add_to_collection('VARIABLES', q)
    g2.add_to_collection('VARIABLES', s)
    g2.add_to_collection('TRAINABLE_VARIABLES', r)    
    
    
print(g1.get_collection('VARIABLES'))
print(g1.get_collection('TRAINABLE_VARIABLES'))
# print(g1.get_collection_ref('VARIABLES'))

print('---------------------------------------')

print(g2.get_collection('VARIABLES'))
print(g2.get_collection('TRAINABLE_VARIABLES'))
    

[<tensorflow.python.ops.variables.Variable object at 0x7f1a2928ee10>, <tensorflow.python.ops.variables.Variable object at 0x7f1a2928ef28>]
[<tf.Tensor 'Mul:0' shape=() dtype=float32>]
---------------------------------------
[<tensorflow.python.ops.variables.Variable object at 0x7f1a2928efd0>, <tensorflow.python.ops.variables.Variable object at 0x7f1a2928ee48>, <tensorflow.python.ops.variables.Variable object at 0x7f1a2928ee80>]
[<tf.Tensor 'Mul:0' shape=() dtype=float32>]


## Validating an object as an elment of a particular Graph:

TensorFlow provides the function tf.Graph.as_graph_element(obj, allow_tensor=True, allow_operation=True),
which can be used to validate that obj represents an element of a particular graph, and gives an informative error message if it is not.

This function returns the Tensor or Operation in the Graph corresponding to 'obj'.

This function raises the following errors:

TypeError: If obj is not a type we support attempting to convert to types.

ValueError: If obj is of an appropriate type but invalid. For example, an invalid string.

KeyError: If obj is not an object in the graph.



In [9]:
g1 = tf.Graph()
with g1.as_default():
    p = tf.Variable(1.0, name="value1")
    q = tf.Variable(2.0, name="value2")
    r = tf.mul(p, q)
    product = tf.identity(r, name="product")

k = tf.Variable(3.0,name="keyError")
    
new_p = g1.as_graph_element(p)
print(new_p)    
    
try:    
    new_error_1 = g1.as_graph_element(a_1)
except NameError as e:
    print(e)
    
try:    
    new_error_2 = g1.as_graph_element(a)
except ValueError as e:
    print(e)

try:    
    new_error_2 = g1.as_graph_element("keyError")
except KeyError as e:
    print(e)
    
    

Tensor("value1:0", shape=(), dtype=float32_ref)
name 'a_1' is not defined
Tensor Tensor("a_1:0", shape=(), dtype=float32) is not an element of this graph.
"The name 'keyError' refers to an Operation not in the graph."


## Finalizing a Graph

We can make a Graph read-only by finalizing it.

tf.Graph.finalize() is used for this purpose.

After making a graph read-only, no new operations can be added to it. This is used to ensure that no operations are added to a graph when it is shared between multiple threads, for example when using a QueueRunner.



In [13]:
g1 = tf.Graph()
with g1.as_default():
    p = tf.Variable(1.0, name="value1")
    q = tf.Variable(2.0, name="value2")
    r = tf.mul(p, q)
    product = tf.identity(r, name="product")
    g1.finalize()
    
    try:
        s = tf.Variable(2.0, name="value3")
    except RuntimeError as e:
        print(e)
    

Graph is finalized and cannot be modified.


## Feeding and Fetching

As we know graph is just the representation of the computations. The real values can be feeded into graph and desired output can be fetch from graphs.

Following are few functions related to these operations:


1) tf.Graph.is_feedable(tensor):

Used to check if a Tensor is feedable or not.
Returns True if and only if tensor is feedable.

2) tf.Graph.is_fetchable(tensor_or_op):

Used to check if a Tensor or an Operation is fetchable or not.
Returns True if and only if tensor_or_op is fetchable.

3) tf.Graph.prevent_feeding(tensor):

Used to mark the given tensor as unfeedable in this graph.

4) tf.Graph.prevent_fetching(op):

Used to mark the given op as unfetchable in this graph.



## 
tf.Graph.get_operations()

Return the list of operations in the graph.

You can modify the operations in place, but modifications to the list such as inserts/delete have no effect on the list of operations known to the graph.

This method may be called concurrently from multiple threads.

Returns:

A list of Operations.